# MCTF: Motion Compensated Temporal Filtering

## Exercise #3 - Multimedia Systems

**Authors:**
- Johan Eduardo Cala Torra
- Alejandro Gonzales Palma

**Institution:** Universidad de Almería (UAL)  
**Course:** Multimedia Systems  
**Date:** February 2026

---

## 1. Theoretical Explanation

### 1.1 What is MCTF?

**Motion Compensated Temporal Filtering (MCTF)** is an advanced video coding technique that exploits temporal correlation between consecutive frames using wavelet transforms and motion compensation. Unlike traditional prediction schemes (such as IPP or IBP), MCTF uses a bidirectional temporal filtering scheme implemented through **lifting**.

### 1.2 Key Differences from Other Schemes

| Feature | IPP (Ex. 1) | IBP (Ex. 2) | **MCTF (Ex. 3)** |
|---------|-------------|-------------|------------------|
| GOP Structure | I-P-P-P... | I-B-P-B-P... | **I-B-B-B...** |
| P-frames | Yes | Yes | **NO** |
| B-frames | No | Yes | Yes (all except I) |
| Method | Unidirectional prediction | Bidirectional prediction | **Temporal filtering with lifting** |
| Structure | Closed-loop | Closed-loop | **Open-loop** |
| Scalability | Limited | Moderate | **High** (temporal, spatial, SNR) |

**Important:** In MCTF there are **NO P-frames**. Only I-frames (first frame of each GOP) and B-frames (all others).

### 1.3 The Lifting Scheme

The **lifting scheme** is an efficient implementation of the Discrete Wavelet Transform (DWT) consisting of three steps:

```
+----------+     +----------+     +----------+
|  Split   | --> | Predict  | --> |  Update  |
+----------+     +----------+     +----------+
```

**Lifting Equations:**

1. **Split:** Separate into even and odd samples
   - Even: `e_i = x_{2i}`
   - Odd: `o_i = x_{2i+1}`

2. **Predict:** Generate high-frequency component (residual)
   - `H_i = o_i - P(e_i)`
   - Where `P(e_i)` is the prediction based on even samples

3. **Update:** Refine low-frequency component
   - `L_i = e_i + U(H_i)`
   - Where `U(H_i)` is the update based on residuals

### 1.4 MCTF Temporal Decomposition

MCTF extends the lifting scheme to the temporal domain with motion compensation:

**MCTF Equations:**

1. **Motion Estimation:** Find motion vectors that minimize error
   - `MV = argmin ||I_t(x,y) - I_{t+1}(x+dx, y+dy)||^2`

2. **Predict Step:** Generate residual using bidirectional MC
   - `H_t = I_odd - (MC(I_prev, MV_bwd) + MC(I_next, MV_fwd))/2 * alpha`

3. **Update Step:** Refine low-pass with residual information
   - `L_t = I_even + MC(H_t, MV_fwd) * beta`

Where `alpha` (predict) and `beta` (update) are wavelet-dependent coefficients.

### 1.5 Supported Wavelet Types

| Wavelet | Predict (alpha) | Update (beta) | Characteristics |
|---------|-----------------|---------------|------------------|
| **Haar** | 1.0 | 0.5 | Simplest, less efficient |
| **5/3 (LeGall)** | 0.5 | 0.25 | Good balance, reversible |
| **9/7 (CDF)** | 1.586 | 0.053 | Best compression, irreversible |

### 1.6 GOP Structure in MCTF

```
GOP with 8 frames:

Original:    F0   F1   F2   F3   F4   F5   F6   F7
Type:        I    B    B    B    B    B    B    B
Level 1:     L0---H0---L1---H1---L2---H2---L3---H3
Level 2:     L0--------H0--------L1--------H1
Level 3:     L0------------------H0
```

- **L frames (Low-pass):** Contain low temporal frequency information
- **H frames (High-pass):** Contain high temporal frequency information (residuals)

### 1.7 References

1. **Gonzalez-Ruiz, V.** "Motion Compensated Temporal Filtering (MCTF)" - https://github.com/vicente-gonzalez-ruiz/motion_compensated_temporal_filtering
2. **Ohm, J.R.** (1994). "Three-dimensional subband coding with motion compensation" - IEEE Transactions on Image Processing
3. **Pesquet-Popescu, B., Bottreau, V.** (2001). "Three-dimensional lifting schemes for motion compensated video compression"
4. **Secker, A., Taubman, D.** (2003). "Lifting-based invertible motion adaptive transform (LIMAT) framework"

---

## 2. Implementation

The MCTF implementation consists of 4 modules:
1. `motion_estimation.py` - Bidirectional motion estimation
2. `motion_compensation.py` - Motion compensation
3. `temporal_filtering.py` - Lifting-based temporal filtering
4. `MCTF.py` - Main codec class

Execute the following cells to generate the implementation files.

### 2.0 Platform Utilities Module

This module provides cross-platform compatibility for file paths and temporary directories.

In [2]:
%%writefile ../src/platform_utils.py
"""
Cross-platform compatibility module for VCF.

This module provides functions and constants to handle file paths
compatible with Windows, Linux, and macOS.
"""

import os
import sys
import tempfile


def get_temp_dir():
    """Get the system temporary directory in a cross-platform way."""
    return tempfile.gettempdir()


def get_vcf_temp_dir():
    """Get the VCF temporary directory, creating it if it doesn't exist."""
    if sys.platform == 'win32':
        temp_dir = "C:/tmp"
    else:
        temp_dir = "/tmp"
    
    os.makedirs(temp_dir, exist_ok=True)
    return temp_dir


def get_temp_path(filename):
    """Build a full path to a file in the VCF temporary directory."""
    return os.path.join(get_vcf_temp_dir(), filename)


def ensure_description_file(description_text):
    """Create the description.txt file required by parser.py."""
    desc_path = get_temp_path("description.txt")
    with open(desc_path, 'w') as f:
        f.write(description_text)


def get_file_uri(path):
    """Convert a file path to file:// URI format."""
    path = os.path.abspath(path)
    if sys.platform == 'win32':
        path = path.replace('\\', '/')
        return f"file:///{path}"
    else:
        return f"file://{path}"


# Predefined path constants
ENCODE_INPUT = "http://www.hpca.ual.es/~vruiz/videos/mobile_352x288x30x420x300.mp4"
ENCODE_OUTPUT_PREFIX = get_temp_path("encoded")
DECODE_INPUT_PREFIX = ENCODE_OUTPUT_PREFIX
DECODE_OUTPUT_PREFIX = get_temp_path("decoded")
DECODE_OUTPUT = get_temp_path("decoded.mp4")

ORIGINAL = get_temp_path("original.png")
ENCODED = get_temp_path("encoded")
DECODED = get_temp_path("decoded.png")

FRAME_PREFIX = get_temp_path("img_")
ORIGINAL_FRAME_PREFIX = get_temp_path("original_")
DECODED_FRAME_PREFIX = get_temp_path("decoded_")


def get_original_frame_path(index, digits=4):
    """Generate path for an original frame."""
    return f"{ORIGINAL_FRAME_PREFIX}{index:0{digits}d}.png"


def get_decoded_frame_path(index, digits=4):
    """Generate path for a decoded frame."""
    return f"{DECODED_FRAME_PREFIX}{index:0{digits}d}.png"


# Ensure temp directory exists on import
_temp_dir = get_vcf_temp_dir()

Overwriting ../src/platform_utils.py


### 2.1 Motion Estimation Module

In [ ]:
%%writefile ../src/motion_estimation.py
"""
Bidirectional motion estimation using block matching.

This module implements bidirectional motion estimation for
MCTF (Motion Compensated Temporal Filtering).

References:
    - Ohm, J.R. (1994). "Three-dimensional subband coding with motion compensation"
      IEEE Transactions on Image Processing, 9:559-571
    - Gonzalez-Ruiz, V. "MCTF"
      https://github.com/vicente-gonzalez-ruiz/motion_compensated_temporal_filtering
"""

import numpy as np
from typing import Tuple
import logging


def block_matching_bidirectional(
    frame_current: np.ndarray,
    frame_prev: np.ndarray,
    frame_next: np.ndarray,
    block_size: int = 16,
    search_range: int = 16
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Bidirectional motion estimation using block matching.
    
    Implements Full Search Block Matching with SAD (Sum of Absolute Differences)
    metric to find best forward and backward motion vectors.
    
    Args:
        frame_current: Current frame (being predicted)
        frame_prev: Previous frame (backward reference)
        frame_next: Next frame (forward reference)
        block_size: Block size NxN (default: 16)
        search_range: Search range +/- pixels (default: 16)
        
    Returns:
        mv_forward: Forward motion vectors (current->next) [blocks_h, blocks_w, 2]
        mv_backward: Backward motion vectors (current->prev) [blocks_h, blocks_w, 2]
    """
    
    height, width = frame_current.shape[:2]
    
    # Number of blocks in each dimension
    blocks_h = height // block_size
    blocks_w = width // block_size
    
    # Initialize motion vector fields
    mv_forward = np.zeros((blocks_h, blocks_w, 2), dtype=np.float32)
    mv_backward = np.zeros((blocks_h, blocks_w, 2), dtype=np.float32)
    
    logging.debug(f"Block matching: {blocks_h}x{blocks_w} blocks, size={block_size}, range={search_range}")
    
    # Iterate over each block
    for by in range(blocks_h):
        for bx in range(blocks_w):
            # Current block coordinates
            y_start = by * block_size
            x_start = bx * block_size
            y_end = y_start + block_size
            x_end = x_start + block_size
            
            # Extract current block
            current_block = frame_current[y_start:y_end, x_start:x_end]
            
            # === Forward search (current -> next) ===
            mv_forward[by, bx] = _search_best_match(
                current_block, frame_next, 
                y_start, x_start, block_size,
                height, width, search_range
            )
            
            # === Backward search (current -> prev) ===
            mv_backward[by, bx] = _search_best_match(
                current_block, frame_prev,
                y_start, x_start, block_size,
                height, width, search_range
            )
    
    return mv_forward, mv_backward


def _search_best_match(
    current_block: np.ndarray,
    reference_frame: np.ndarray,
    y_start: int,
    x_start: int,
    block_size: int,
    height: int,
    width: int,
    search_range: int
) -> Tuple[float, float]:
    """
    Search for best match of a block in reference frame.
    
    Args:
        current_block: Block to search
        reference_frame: Frame to search in
        y_start, x_start: Block position in original frame
        block_size: Block size
        height, width: Frame dimensions
        search_range: Search range
        
    Returns:
        (dx, dy): Motion vector that minimizes SAD
    """
    
    min_sad = float('inf')
    best_mv = (0, 0)
    
    for dy in range(-search_range, search_range + 1):
        for dx in range(-search_range, search_range + 1):
            # Coordinates in reference frame
            ref_y = y_start + dy
            ref_x = x_start + dx
            
            # Check boundaries
            if (ref_y >= 0 and ref_y + block_size <= height and
                ref_x >= 0 and ref_x + block_size <= width):
                
                # Extract reference block
                ref_block = reference_frame[
                    ref_y:ref_y + block_size,
                    ref_x:ref_x + block_size
                ]
                
                # Calculate SAD (Sum of Absolute Differences) - vectorized
                sad = np.sum(np.abs(
                    current_block.astype(np.float32) - 
                    ref_block.astype(np.float32)
                ))
                
                # Update best match
                if sad < min_sad:
                    min_sad = sad
                    best_mv = (dx, dy)
    
    return best_mv

### 2.2 Motion Compensation Module

In [ ]:
%%writefile ../src/motion_compensation.py
"""
Motion compensation for MCTF.

This module implements motion compensation that applies motion vectors
to reference frames to generate predictions.

References:
    - Gonzalez-Ruiz, V. "Motion Compensation"
      https://github.com/vicente-gonzalez-ruiz/motion_compensation
    - Pesquet-Popescu, B., Bottreau, V. (2001). "Three-dimensional lifting 
      schemes for motion compensated video compression"
"""

import numpy as np
import logging


def motion_compensate(
    frame: np.ndarray,
    motion_vectors: np.ndarray,
    block_size: int = 16
) -> np.ndarray:
    """
    Apply motion compensation to a frame.
    
    Generates a compensated frame by shifting blocks according to
    the provided motion vectors.
    
    Args:
        frame: Reference frame
        motion_vectors: Motion vector field [blocks_h, blocks_w, 2]
        block_size: Block size (default: 16)
        
    Returns:
        compensated_frame: Motion compensated frame
    """
    
    height, width = frame.shape[:2]
    compensated_frame = np.zeros_like(frame, dtype=np.float32)
    
    blocks_h, blocks_w = motion_vectors.shape[:2]
    
    logging.debug(f"Motion compensate: {blocks_h}x{blocks_w} blocks, size={block_size}")
    
    for by in range(blocks_h):
        for bx in range(blocks_w):
            # Destination block coordinates
            y_start = by * block_size
            x_start = bx * block_size
            y_end = min(y_start + block_size, height)
            x_end = min(x_start + block_size, width)
            
            # Motion vector
            dx, dy = motion_vectors[by, bx]
            
            # Coordinates in reference frame
            ref_y = int(y_start + dy)
            ref_x = int(x_start + dx)
            
            # Check boundaries
            if (ref_y >= 0 and ref_y + block_size <= height and
                ref_x >= 0 and ref_x + block_size <= width):
                
                # Copy compensated block
                compensated_frame[y_start:y_end, x_start:x_end] = \
                    frame[ref_y:ref_y + (y_end - y_start), 
                          ref_x:ref_x + (x_end - x_start)]
            else:
                # If out of bounds, copy original block
                compensated_frame[y_start:y_end, x_start:x_end] = \
                    frame[y_start:y_end, x_start:x_end]
    
    return compensated_frame


def motion_compensate_bidirectional(
    frame_prev: np.ndarray,
    frame_next: np.ndarray,
    mv_backward: np.ndarray,
    mv_forward: np.ndarray,
    block_size: int = 16
) -> np.ndarray:
    """
    Bidirectional motion compensation for B-frames.
    
    Generates a bidirectional prediction by averaging compensations
    from previous and next frames.
    
    Args:
        frame_prev: Previous frame (backward reference)
        frame_next: Next frame (forward reference)
        mv_backward: Backward motion vectors
        mv_forward: Forward motion vectors
        block_size: Block size (default: 16)
        
    Returns:
        prediction: Bidirectional prediction frame
    """
    
    # Compensate from previous frame
    mc_prev = motion_compensate(frame_prev, mv_backward, block_size)
    
    # Compensate from next frame
    mc_next = motion_compensate(frame_next, mv_forward, block_size)
    
    # Bidirectional prediction (average)
    prediction = (mc_prev + mc_next) / 2.0
    
    return prediction

### 2.3 Temporal Filtering Module

In [ ]:
%%writefile ../src/temporal_filtering.py
"""
Temporal filtering using lifting scheme with motion compensation.

This module implements temporal wavelet filtering for MCTF using
the lifting scheme with Predict and Update steps.

References:
    - Pesquet-Popescu, B., Bottreau, V. (2001). "Three-dimensional lifting 
      schemes for motion compensated video compression"
    - Secker, A., Taubman, D. (2003). "Lifting-based invertible motion 
      adaptive transform (LIMAT) framework"
    - Gonzalez-Ruiz, V. "MCTF"
      https://github.com/vicente-gonzalez-ruiz/motion_compensated_temporal_filtering
"""

import numpy as np
from typing import List, Tuple
import logging

from motion_compensation import motion_compensate


# Wavelet coefficients for lifting scheme
WAVELET_COEFFICIENTS = {
    'haar': {
        'predict': 1.0,
        'update': 0.5
    },
    '5/3': {
        'predict': 0.5,
        'update': 0.25
    },
    '9/7': {
        'predict': 1.586134342,
        'update': 0.052980118
    }
}


def get_wavelet_coefficients(wavelet_type: str) -> Tuple[float, float]:
    """
    Get predict and update coefficients for a wavelet.
    
    Args:
        wavelet_type: Wavelet type ('haar', '5/3', '9/7')
        
    Returns:
        (predict_coef, update_coef): Lifting scheme coefficients
        
    Raises:
        ValueError: If wavelet type is not supported
    """
    if wavelet_type not in WAVELET_COEFFICIENTS:
        raise ValueError(
            f"Wavelet type '{wavelet_type}' not supported. "
            f"Supported types: {list(WAVELET_COEFFICIENTS.keys())}"
        )
    
    coeffs = WAVELET_COEFFICIENTS[wavelet_type]
    return coeffs['predict'], coeffs['update']


def temporal_filter_lifting(
    frames: List[np.ndarray],
    motion_vectors_forward: List[np.ndarray],
    motion_vectors_backward: List[np.ndarray],
    wavelet_type: str = '5/3',
    block_size: int = 16
) -> Tuple[List[np.ndarray], List[np.ndarray]]:
    """
    Apply temporal filtering using lifting scheme with motion compensation.
    
    Implements IBB... scheme where even frames are low-pass (L)
    and odd frames generate high-pass (H) as prediction residuals.
    
    Args:
        frames: List of frames to filter
        motion_vectors_forward: List of forward MVs
        motion_vectors_backward: List of backward MVs
        wavelet_type: Wavelet type ('haar', '5/3', '9/7')
        block_size: Block size for MC
        
    Returns:
        low_pass: Low temporal frequency frames (L)
        high_pass: High temporal frequency frames (H/residuals)
    """
    
    n_frames = len(frames)
    predict_coef, update_coef = get_wavelet_coefficients(wavelet_type)
    
    logging.info(f"Temporal filtering {n_frames} frames with {wavelet_type} wavelet")
    
    low_pass = []
    high_pass = []
    
    # Process frame pairs (even, odd)
    for i in range(0, n_frames - 1, 2):
        frame_even = frames[i].astype(np.float32)      # Even frame (t=0,2,4...)
        frame_odd = frames[i + 1].astype(np.float32)   # Odd frame (t=1,3,5...)
        
        # === PREDICT STEP ===
        # Predict odd frame using MC from neighboring even frames
        
        # MC from previous frame (current even)
        mc_prev = motion_compensate(
            frame_even,
            motion_vectors_backward[i] if i < len(motion_vectors_backward) else np.zeros_like(motion_vectors_backward[0]),
            block_size
        )
        
        # MC from next frame (even i+2) if exists
        if i + 2 < n_frames:
            mc_next = motion_compensate(
                frames[i + 2].astype(np.float32),
                motion_vectors_forward[i + 1] if i + 1 < len(motion_vectors_forward) else np.zeros_like(motion_vectors_forward[0]),
                block_size
            )
        else:
            mc_next = mc_prev
        
        # Bidirectional prediction
        prediction = (mc_prev + mc_next) * predict_coef / 2.0
        
        # High frequency residual (H)
        h_frame = frame_odd - prediction
        high_pass.append(h_frame)
        
        # === UPDATE STEP ===
        # Update even frame with residual information
        mc_residual = motion_compensate(
            h_frame,
            motion_vectors_forward[i] if i < len(motion_vectors_forward) else np.zeros_like(motion_vectors_forward[0]),
            block_size
        )
        
        # Update (L)
        l_frame = frame_even + mc_residual * update_coef
        low_pass.append(l_frame)
    
    # If odd number of frames, last one passes as low-pass
    if n_frames % 2 != 0:
        low_pass.append(frames[-1].astype(np.float32))
    
    logging.info(f"Temporal filtering complete: {len(low_pass)} L frames, {len(high_pass)} H frames")

    return low_pass, high_pass


def inverse_temporal_filter_lifting(
    low_pass: List[np.ndarray],
    high_pass: List[np.ndarray],
    motion_vectors_forward: List[np.ndarray],
    motion_vectors_backward: List[np.ndarray],
    wavelet_type: str = '5/3',
    block_size: int = 16
) -> List[np.ndarray]:
    """
    Reconstruct frames from temporal decomposition.

    Applies inverse lifting scheme to recover original frames
    from L (low-pass) and H (high-pass) components.

    Args:
        low_pass: L frames (low temporal frequency)
        high_pass: H frames (high temporal frequency)
        motion_vectors_forward: Forward MVs
        motion_vectors_backward: Backward MVs
        wavelet_type: Wavelet type used in encoding
        block_size: Block size for MC

    Returns:
        reconstructed_frames: List of reconstructed frames
    """

    predict_coef, update_coef = get_wavelet_coefficients(wavelet_type)

    n_low = len(low_pass)
    n_high = len(high_pass)

    logging.info(f"Inverse temporal filtering: {n_low} L frames, {n_high} H frames")

    reconstructed_frames = []

    for i in range(n_high):
        l_frame = low_pass[i].astype(np.float32)
        h_frame = high_pass[i].astype(np.float32)

        # === INVERSE UPDATE ===
        # Recover original even frame
        mc_residual = motion_compensate(
            h_frame,
            motion_vectors_forward[2 * i] if 2 * i < len(motion_vectors_forward) else np.zeros_like(motion_vectors_forward[0]),
            block_size
        )

        frame_even = l_frame - mc_residual * update_coef
        reconstructed_frames.append(frame_even)

        # === INVERSE PREDICT ===
        # Recover original odd frame

        # MC for prediction
        mc_prev = motion_compensate(
            frame_even,
            motion_vectors_backward[2 * i] if 2 * i < len(motion_vectors_backward) else np.zeros_like(motion_vectors_backward[0]),
            block_size
        )

        if i + 1 < n_low:
            mc_next = motion_compensate(
                low_pass[i + 1].astype(np.float32),
                motion_vectors_forward[2 * i + 1] if 2 * i + 1 < len(motion_vectors_forward) else np.zeros_like(motion_vectors_forward[0]),
                block_size
            )
        else:
            mc_next = mc_prev

        # Bidirectional prediction
        prediction = (mc_prev + mc_next) * predict_coef / 2.0

        # Inverse predict
        frame_odd = h_frame + prediction
        reconstructed_frames.append(frame_odd)

    # If there was an extra frame in low_pass (odd number of frames)
    if n_low > n_high:
        reconstructed_frames.append(low_pass[-1].astype(np.float32))

    logging.info(f"Inverse temporal filtering complete: {len(reconstructed_frames)} frames")

    return reconstructed_frames

### 2.4 Main MCTF Codec Module

In [ ]:
%%writefile ../src/MCTF.py
"""
MCTF: Motion Compensated Temporal Filtering codec.

This module implements a video codec based on temporal filtering
with motion compensation using lifting scheme.

GOP structure: IBB... (I-frame followed by B-frames, NO P-frames)
- I-frame: coded independently (intra)
- B-frames: bidirectionally predicted

References:
    - Ohm, J.R. (1994). "Three-dimensional subband coding with motion compensation"
    - Pesquet-Popescu, B., Bottreau, V. (2001). "Three-dimensional lifting schemes"
    - Gonzalez-Ruiz, V. "MCTF" https://github.com/vicente-gonzalez-ruiz/motion_compensated_temporal_filtering
"""

import sys
import os
import logging
import numpy as np
import cv2
import av
from PIL import Image
import importlib
import pickle
import tempfile

import main
import platform_utils as pu

# Cross-platform initialization
TMP_DIR = pu.get_vcf_temp_dir()
pu.ensure_description_file(__doc__)

import parser
import entropy_video_coding as EVC

from motion_estimation import block_matching_bidirectional
from motion_compensation import motion_compensate
from temporal_filtering import temporal_filter_lifting, inverse_temporal_filter_lifting

# Default parameters
DEFAULT_GOP_SIZE = 16
DEFAULT_TEMPORAL_LEVELS = 4
DEFAULT_BLOCK_SIZE = 16
DEFAULT_SEARCH_RANGE = 16
DEFAULT_WAVELET_TYPE = '5/3'

# Encoder parser
parser.parser_encode.add_argument("-V", "--video_input", type=parser.int_or_str,
    help=f"Input video (default: {EVC.ENCODE_INPUT})",
    default=EVC.ENCODE_INPUT)
parser.parser_encode.add_argument("-O", "--video_output", type=parser.int_or_str,
    help=f"Output prefix (default: {EVC.ENCODE_OUTPUT_PREFIX})",
    default=EVC.ENCODE_OUTPUT_PREFIX)
parser.parser_encode.add_argument("-T", "--transform", type=str,
    help=f"2D-transform (default: {EVC.DEFAULT_TRANSFORM})",
    default=EVC.DEFAULT_TRANSFORM)
parser.parser_encode.add_argument("-N", "--number_of_frames", type=parser.int_or_str,
    help=f"Number of frames to encode (default: {EVC.N_FRAMES})",
    default=f"{EVC.N_FRAMES}")
parser.parser_encode.add_argument("--gop_size", type=int,
    help=f"GOP size (default: {DEFAULT_GOP_SIZE})",
    default=DEFAULT_GOP_SIZE)
parser.parser_encode.add_argument("--temporal_levels", type=int,
    help=f"Temporal decomposition levels (default: {DEFAULT_TEMPORAL_LEVELS})",
    default=DEFAULT_TEMPORAL_LEVELS)
parser.parser_encode.add_argument("--block_size", type=int,
    help=f"Block size for motion estimation (default: {DEFAULT_BLOCK_SIZE})",
    default=DEFAULT_BLOCK_SIZE)
parser.parser_encode.add_argument("--search_range", type=int,
    help=f"Search range for motion estimation (default: {DEFAULT_SEARCH_RANGE})",
    default=DEFAULT_SEARCH_RANGE)
parser.parser_encode.add_argument("--wavelet_type", type=str,
    help=f"Temporal wavelet type: haar, 5/3, 9/7 (default: {DEFAULT_WAVELET_TYPE})",
    default=DEFAULT_WAVELET_TYPE)

# Decoder parser
parser.parser_decode.add_argument("-V", "--video_input", type=parser.int_or_str,
    help=f"Input MCTF stream prefix (default: {EVC.ENCODE_OUTPUT_PREFIX})",
    default=EVC.ENCODE_OUTPUT_PREFIX)
parser.parser_decode.add_argument("-O", "--video_output", type=parser.int_or_str,
    help=f"Output prefix (default: {EVC.DECODE_OUTPUT_PREFIX})",
    default=EVC.DECODE_OUTPUT_PREFIX)
parser.parser_decode.add_argument("-T", "--transform", type=str,
    help=f"2D-transform (default: {EVC.DEFAULT_TRANSFORM})",
    default=EVC.DEFAULT_TRANSFORM)
parser.parser_decode.add_argument("-N", "--number_of_frames", type=parser.int_or_str,
    help=f"Number of frames to decode (default: {EVC.N_FRAMES})",
    default=f"{EVC.N_FRAMES}")
parser.parser_decode.add_argument("--gop_size", type=int,
    help=f"GOP size (default: {DEFAULT_GOP_SIZE})",
    default=DEFAULT_GOP_SIZE)
parser.parser_decode.add_argument("--temporal_levels", type=int,
    help=f"Temporal decomposition levels (default: {DEFAULT_TEMPORAL_LEVELS})",
    default=DEFAULT_TEMPORAL_LEVELS)
parser.parser_decode.add_argument("--block_size", type=int,
    help=f"Block size for motion estimation (default: {DEFAULT_BLOCK_SIZE})",
    default=DEFAULT_BLOCK_SIZE)
parser.parser_decode.add_argument("--search_range", type=int,
    help=f"Search range for motion estimation (default: {DEFAULT_SEARCH_RANGE})",
    default=DEFAULT_SEARCH_RANGE)
parser.parser_decode.add_argument("--wavelet_type", type=str,
    help=f"Temporal wavelet type: haar, 5/3, 9/7 (default: {DEFAULT_WAVELET_TYPE})",
    default=DEFAULT_WAVELET_TYPE)

args = parser.parser.parse_known_args()[0]

# Import spatial transform
if __debug__:
    if args.debug:
        print(f"MCTF: Importing {args.transform}")

try:
    transform = importlib.import_module(args.transform)
except ImportError as e:
    print(f"Error: Could not find {args.transform} module ({e})")
    sys.exit(1)


class CoDec(EVC.CoDec):
    """
    MCTF (Motion Compensated Temporal Filtering) Codec.
    
    Implements video compression using:
    1. Bidirectional motion estimation
    2. Temporal filtering with lifting scheme (Predict + Update)
    3. 2D spatial transform (DCT or DWT)
    4. Quantization and entropy coding
    """

    def __init__(self, args):
        logging.debug("trace")
        super().__init__(args)
        
        # MCTF configuration
        self.gop_size = args.gop_size
        self.temporal_levels = args.temporal_levels
        self.block_size = args.block_size
        self.search_range = args.search_range
        self.wavelet_type = args.wavelet_type
        
        # Spatial transform codec
        self.transform_codec = transform.CoDec(args)
        
        logging.info(f"MCTF Codec initialized:")
        logging.info(f"  GOP size: {self.gop_size}")
        logging.info(f"  Temporal levels: {self.temporal_levels}")
        logging.info(f"  Block size: {self.block_size}")
        logging.info(f"  Search range: {self.search_range}")
        logging.info(f"  Wavelet type: {self.wavelet_type}")
        logging.info(f"  Spatial transform: {args.transform}")

    def bye(self):
        """Override bye() to use video_input/video_output."""
        logging.debug("trace")
        if __debug__:
            if self.encoding:
                BPP = (self.total_output_size*8)/(self.N_frames*self.width*self.height)
                logging.info(f"Output bit-rate = {BPP} bits/pixel")
                # Save metadata
                with open(f"{self.args.video_output}.txt", 'w') as f:
                    f.write(f"{self.args.video_input}\n")
                    f.write(f"{self.N_frames}\n")
                    f.write(f"{self.height}\n")
                    f.write(f"{self.width}\n")
                    f.write(f"{BPP}\n")
            else:
                # Read metadata and calculate distortion
                with open(f"{self.args.video_input}.txt", 'r') as f:
                    original_file = f.readline().strip()
                    logging.info(f"original_file = {original_file}")
                    N_frames = int(f.readline().strip())
                    logging.info(f"N_frames = {N_frames}")
                    height = f.readline().strip()
                    logging.info(f"video height = {height} pixels")
                    width = f.readline().strip()
                    logging.info(f"video width = {width} pixels")
                    BPP = float(f.readline().strip())
                    logging.info(f"BPP = {BPP}")

    def encode(self):
        """
        Encode a video using MCTF.

        Process:
        1. Read frames from input video
        2. Group frames into GOPs
        3. For each GOP:
           a. Estimate bidirectional motion
           b. Apply temporal filtering (lifting)
           c. Encode L and H frames with spatial transform
        """
        logging.debug("trace")
        fn = self.args.video_input
        logging.info(f"MCTF Encoding {fn}")

        # Read video and extract frames
        container = av.open(fn)
        frames = []
        img_counter = 0

        for packet in container.demux():
            if __debug__:
                self.total_input_size += packet.size
            for frame in packet.decode():
                img = frame.to_image()
                img_array = np.array(img.convert('L'))  # Grayscale for MCTF
                frames.append(img_array)

                # Save original for comparison
                if __debug__:
                    img_fn = os.path.join(TMP_DIR, f"original_{img_counter:04d}.png")
                    img.save(img_fn)

                img_counter += 1
                if img_counter >= self.args.number_of_frames:
                    break
            if img_counter >= self.args.number_of_frames:
                break

        self.N_frames = len(frames)
        self.height, self.width = frames[0].shape
        logging.info(f"Read {self.N_frames} frames of size {self.width}x{self.height}")

        # Process GOPs
        gop_data_list = []
        frame_idx = 0
        gop_counter = 0

        while frame_idx < self.N_frames:
            gop_end = min(frame_idx + self.gop_size, self.N_frames)
            gop_frames = frames[frame_idx:gop_end]

            logging.info(f"Processing GOP {gop_counter}: frames {frame_idx}-{gop_end-1}")

            # Encode GOP
            gop_data = self._encode_gop(gop_frames, gop_counter)
            gop_data_list.append(gop_data)

            frame_idx = gop_end
            gop_counter += 1

        # Write encoded stream
        self._write_mctf_stream(gop_data_list)

        logging.info(f"MCTF encoding complete: {gop_counter} GOPs")

    def _encode_gop(self, gop_frames, gop_idx):
        """Encode a GOP using MCTF."""
        n_frames = len(gop_frames)

        if n_frames < 2:
            # Very small GOP: encode as intra only
            return self._encode_intra_only(gop_frames, gop_idx)

        # === Step 1: Motion estimation ===
        logging.info(f"  Motion estimation for {n_frames} frames")
        mv_forward_list = []
        mv_backward_list = []

        for i in range(n_frames):
            frame_current = gop_frames[i]
            frame_prev = gop_frames[max(0, i - 1)]
            frame_next = gop_frames[min(n_frames - 1, i + 1)]

            mv_fwd, mv_bwd = block_matching_bidirectional(
                frame_current, frame_prev, frame_next,
                self.block_size, self.search_range
            )
            mv_forward_list.append(mv_fwd)
            mv_backward_list.append(mv_bwd)

        # === Step 2: Temporal filtering (lifting) ===
        logging.info(f"  Temporal filtering with {self.wavelet_type} wavelet")
        low_pass, high_pass = temporal_filter_lifting(
            gop_frames,
            mv_forward_list,
            mv_backward_list,
            self.wavelet_type,
            self.block_size
        )

        # === Step 3: Encode L and H frames with spatial transform ===
        logging.info(f"  Encoding {len(low_pass)} L frames and {len(high_pass)} H frames")

        encoded_low = []
        for i, l_frame in enumerate(low_pass):
            l_frame_uint8 = np.clip(l_frame, 0, 255).astype(np.uint8)
            fn_l = os.path.join(TMP_DIR, f"gop{gop_idx:02d}_L_{i:02d}")
            self._save_and_encode_frame(l_frame_uint8, fn_l)
            encoded_low.append(fn_l)

        encoded_high = []
        for i, h_frame in enumerate(high_pass):
            # Residuals can be negative: offset to positive
            h_frame_offset = h_frame + 128
            h_frame_uint8 = np.clip(h_frame_offset, 0, 255).astype(np.uint8)
            fn_h = os.path.join(TMP_DIR, f"gop{gop_idx:02d}_H_{i:02d}")
            self._save_and_encode_frame(h_frame_uint8, fn_h)
            encoded_high.append(fn_h)

        return {
            'n_frames': n_frames,
            'mv_forward': mv_forward_list,
            'mv_backward': mv_backward_list,
            'encoded_low': encoded_low,
            'encoded_high': encoded_high,
            'wavelet_type': self.wavelet_type
        }

    def _encode_intra_only(self, frames, gop_idx):
        """Encode frames as intra only."""
        encoded_frames = []
        for i, frame in enumerate(frames):
            fn = os.path.join(TMP_DIR, f"gop{gop_idx:02d}_I_{i:02d}")
            self._save_and_encode_frame(frame, fn)
            encoded_frames.append(fn)

        return {
            'n_frames': len(frames),
            'intra_only': True,
            'encoded_frames': encoded_frames
        }

    def _save_and_encode_frame(self, frame, fn_prefix):
        """Save and encode a frame using TIFF with lossless compression."""
        # Convert to RGB if grayscale
        if len(frame.shape) == 2:
            frame_rgb = np.stack([frame] * 3, axis=-1)
        else:
            frame_rgb = frame

        # Save as TIFF with lossless compression
        img_fn = f"{fn_prefix}.tif"
        img = Image.fromarray(frame_rgb)
        img.save(img_fn, compression='tiff_deflate')

        output_size = os.path.getsize(img_fn)
        self.total_output_size += output_size
        return output_size

    def _write_mctf_stream(self, gop_data_list):
        """Write the encoded MCTF stream."""
        stream_fn = f"{self.args.video_output}.mctf"

        header = {
            'n_frames': self.N_frames,
            'height': self.height,
            'width': self.width,
            'gop_size': self.gop_size,
            'temporal_levels': self.temporal_levels,
            'block_size': self.block_size,
            'wavelet_type': self.wavelet_type,
            'num_gops': len(gop_data_list)
        }

        with open(stream_fn, 'wb') as f:
            pickle.dump(header, f)
            for gop_data in gop_data_list:
                pickle.dump(gop_data, f)

        stream_size = os.path.getsize(stream_fn)
        self.total_output_size += stream_size
        logging.info(f"Written MCTF stream: {stream_fn} ({stream_size} bytes)")

    def decode(self):
        """
        Decode a video encoded with MCTF.

        Inverse process:
        1. Read MCTF stream
        2. For each GOP:
           a. Decode L and H frames with inverse spatial transform
           b. Apply inverse temporal filtering (inverse lifting)
        3. Write decoded frames
        """
        logging.debug("trace")
        logging.info(f"MCTF Decoding {self.args.video_input}")

        # Read MCTF stream
        stream_fn = f"{self.args.video_input}.mctf"
        header, gop_data_list = self._read_mctf_stream(stream_fn)

        self.N_frames = header['n_frames']
        self.height = header['height']
        self.width = header['width']

        logging.info(f"Decoding {self.N_frames} frames of size {self.width}x{self.height}")
        logging.info(f"  {header['num_gops']} GOPs")

        # Decode each GOP
        all_frames = []
        for gop_idx, gop_data in enumerate(gop_data_list):
            logging.info(f"Decoding GOP {gop_idx}")

            if gop_data.get('intra_only', False):
                gop_frames = self._decode_intra_only(gop_data)
            else:
                gop_frames = self._decode_gop(gop_data)

            all_frames.extend(gop_frames)

        # Write decoded frames
        for i, frame in enumerate(all_frames):
            frame_uint8 = np.clip(frame, 0, 255).astype(np.uint8)
            out_fn = os.path.join(TMP_DIR, f"decoded_{i:04d}.png")

            if len(frame_uint8.shape) == 2:
                frame_rgb = np.stack([frame_uint8] * 3, axis=-1)
            else:
                frame_rgb = frame_uint8

            Image.fromarray(frame_rgb).save(out_fn)
            logging.info(f"Decoded frame {i} to {out_fn}")

        logging.info(f"MCTF decoding complete: {len(all_frames)} frames")

    def _read_mctf_stream(self, stream_fn):
        """Read the encoded MCTF stream."""
        with open(stream_fn, 'rb') as f:
            header = pickle.load(f)
            gop_data_list = []
            for _ in range(header['num_gops']):
                gop_data = pickle.load(f)
                gop_data_list.append(gop_data)

        return header, gop_data_list

    def _decode_gop(self, gop_data):
        """Decode a GOP using inverse MCTF."""
        n_frames = gop_data['n_frames']
        mv_forward = gop_data['mv_forward']
        mv_backward = gop_data['mv_backward']
        wavelet_type = gop_data['wavelet_type']

        # === Step 1: Decode L and H frames ===
        logging.info(f"  Decoding L and H frames")

        low_pass = []
        for fn_l in gop_data['encoded_low']:
            frame = self._decode_frame(fn_l)
            low_pass.append(frame.astype(np.float32))

        high_pass = []
        for fn_h in gop_data['encoded_high']:
            frame = self._decode_frame(fn_h)
            # Remove offset added during encoding
            frame_float = frame.astype(np.float32) - 128
            high_pass.append(frame_float)

        # === Step 2: Inverse temporal filtering ===
        logging.info(f"  Inverse temporal filtering with {wavelet_type} wavelet")
        reconstructed = inverse_temporal_filter_lifting(
            low_pass,
            high_pass,
            mv_forward,
            mv_backward,
            wavelet_type,
            self.block_size
        )

        return reconstructed

    def _decode_intra_only(self, gop_data):
        """Decode intra-only frames."""
        frames = []
        for fn in gop_data['encoded_frames']:
            frame = self._decode_frame(fn)
            frames.append(frame.astype(np.float32))
        return frames

    def _decode_frame(self, fn_prefix):
        """Decode a frame by reading the TIFF."""
        img_fn = f"{fn_prefix}.tif"

        # Read frame from TIFF
        img = Image.open(img_fn)
        frame = np.array(img.convert('L'))  # Grayscale
        return frame


if __name__ == "__main__":
    main.main(parser.parser, logging, CoDec)

---

## 3. Usage within VCF

This section demonstrates how to use the MCTF codec within the VCF framework.

### 3.1 View Available Options

In [1]:
# View available command-line options
!python ../src/MCTF.py -h

usage: MCTF.py [-h] [-g] {encode,decode} ...

positional arguments:
  {encode,decode}  You must specify one of the following subcomands:
    encode         Compress data
    decode         Uncompress data

options:
  -h, --help       show this help message and exit
  -g, --debug      Output debug information (default: False)


In [2]:
# View encoding options
!python ../src/MCTF.py encode -h

usage: MCTF.py encode [-h] [-V VIDEO_INPUT] [-O VIDEO_OUTPUT] [-T TRANSFORM]
                      [-N NUMBER_OF_FRAMES] [--gop_size GOP_SIZE]
                      [--temporal_levels TEMPORAL_LEVELS]
                      [--block_size BLOCK_SIZE] [--search_range SEARCH_RANGE]
                      [--wavelet_type WAVELET_TYPE]

options:
  -h, --help            show this help message and exit
  -V VIDEO_INPUT, --video_input VIDEO_INPUT
                        Input video (default: http://www.hpca.ual.es/~vruiz/vi
                        deos/mobile_352x288x30x420x300.mp4)
  -O VIDEO_OUTPUT, --video_output VIDEO_OUTPUT
                        Output prefix (default: C:/tmp\encoded)
  -T TRANSFORM, --transform TRANSFORM
                        2D-transform (default: 2D-DCT)
  -N NUMBER_OF_FRAMES, --number_of_frames NUMBER_OF_FRAMES
                        Number of frames to encode (default: 3)
  --gop_size GOP_SIZE   GOP size (default: 16)
  --temporal_levels TEMPORAL_LEVELS
          

In [3]:
# View decoding options
!python ../src/MCTF.py decode -h

usage: MCTF.py decode [-h] [-V VIDEO_INPUT] [-O VIDEO_OUTPUT] [-T TRANSFORM]
                      [-N NUMBER_OF_FRAMES] [--gop_size GOP_SIZE]
                      [--temporal_levels TEMPORAL_LEVELS]
                      [--block_size BLOCK_SIZE] [--search_range SEARCH_RANGE]
                      [--wavelet_type WAVELET_TYPE]

options:
  -h, --help            show this help message and exit
  -V VIDEO_INPUT, --video_input VIDEO_INPUT
                        Input MCTF stream prefix (default: C:/tmp\encoded)
  -O VIDEO_OUTPUT, --video_output VIDEO_OUTPUT
                        Output prefix (default: C:/tmp\decoded)
  -T TRANSFORM, --transform TRANSFORM
                        2D-transform (default: 2D-DCT)
  -N NUMBER_OF_FRAMES, --number_of_frames NUMBER_OF_FRAMES
                        Number of frames to decode (default: 3)
  --gop_size GOP_SIZE   GOP size (default: 16)
  --temporal_levels TEMPORAL_LEVELS
                        Temporal decomposition levels (default: 4)
  --blo

### 3.2 Encoding a Video

The following cell encodes a video using MCTF with default parameters.

In [4]:
# Encode 16 frames with default parameters
# (GOP size=16, block_size=16, wavelet=5/3)
!python ../src/MCTF.py encode -N 16

main Namespace(debug=False, subparser_name='encode', video_input='http://www.hpca.ual.es/~vruiz/videos/mobile_352x288x30x420x300.mp4', video_output='C:/tmp\\encoded', transform='2D-DCT', number_of_frames=16, gop_size=16, temporal_levels=4, block_size=16, search_range=16, wavelet_type='5/3', block_size_DCT=8, color_transform='YCoCg', perceptual_quantization=False, Lambda=None, disable_subbands=False, quantizer='deadzone', QSS=32, entropy_image_codec='TIFF', original='C:/tmp\\original.png', encoded='C:/tmp\\encoded', func=<function encode at 0x0000027F779C0FE0>)


(INFO) MCTF: MCTF Codec initialized:
(INFO) MCTF:   GOP size: 16
(INFO) MCTF:   Temporal levels: 4
(INFO) MCTF:   Block size: 16
(INFO) MCTF:   Search range: 16
(INFO) MCTF:   Wavelet type: 5/3
(INFO) MCTF:   Spatial transform: 2D-DCT
(INFO) MCTF: MCTF Encoding http://www.hpca.ual.es/~vruiz/videos/mobile_352x288x30x420x300.mp4
(INFO) MCTF: Read 16 frames of size 352x288
(INFO) MCTF: Processing GOP 0: frames 0-15
(INFO) MCTF:   Motion estimation for 16 frames
(INFO) MCTF:   Temporal filtering with 5/3 wavelet
(INFO) temporal_filtering: Temporal filtering 16 frames with 5/3 wavelet
(INFO) temporal_filtering: Temporal filtering complete: 8 L frames, 8 H frames
(INFO) MCTF:   Encoding 8 L frames and 8 H frames
(INFO) MCTF: Written MCTF stream: C:/tmp\encoded.mctf (103251 bytes)
(INFO) MCTF: MCTF encoding complete: 1 GOPs
(INFO) MCTF: Output bit-rate = 13.090312302714647 bits/pixel


### 3.3 Encoding with Custom Parameters

In [6]:
# Encode with custom parameters:
# - 8 frames
# - GOP size of 8
# - Block size of 8 (smaller blocks for finer motion)
# - Search range of 32 (larger search area)
# - 9/7 wavelet (better compression, irreversible)
!python ../src/MCTF.py encode -N 8 --gop_size 8 --block_size 8 --search_range 32 --wavelet_type 9/7

main Namespace(debug=False, subparser_name='encode', video_input='http://www.hpca.ual.es/~vruiz/videos/mobile_352x288x30x420x300.mp4', video_output='C:/tmp\\encoded', transform='2D-DCT', number_of_frames=8, gop_size=8, temporal_levels=4, block_size=8, search_range=32, wavelet_type='9/7', block_size_DCT=8, color_transform='YCoCg', perceptual_quantization=False, Lambda=None, disable_subbands=False, quantizer='deadzone', QSS=32, entropy_image_codec='TIFF', original='C:/tmp\\original.png', encoded='C:/tmp\\encoded', func=<function encode at 0x000002BB97E90FE0>)


(INFO) MCTF: MCTF Codec initialized:
(INFO) MCTF:   GOP size: 8
(INFO) MCTF:   Temporal levels: 4
(INFO) MCTF:   Block size: 8
(INFO) MCTF:   Search range: 32
(INFO) MCTF:   Wavelet type: 9/7
(INFO) MCTF:   Spatial transform: 2D-DCT
(INFO) MCTF: MCTF Encoding http://www.hpca.ual.es/~vruiz/videos/mobile_352x288x30x420x300.mp4
(INFO) MCTF: Read 8 frames of size 352x288
(INFO) MCTF: Processing GOP 0: frames 0-7
(INFO) MCTF:   Motion estimation for 8 frames
(INFO) MCTF:   Temporal filtering with 9/7 wavelet
(INFO) temporal_filtering: Temporal filtering 8 frames with 9/7 wavelet
(INFO) temporal_filtering: Temporal filtering complete: 4 L frames, 4 H frames
(INFO) MCTF:   Encoding 4 L frames and 4 H frames
(INFO) MCTF: Written MCTF stream: C:/tmp\encoded.mctf (203884 bytes)
(INFO) MCTF: MCTF encoding complete: 1 GOPs
(INFO) MCTF: Output bit-rate = 14.383897569444445 bits/pixel


### 3.4 Decoding

In [7]:
# Decode the encoded video
!python ../src/MCTF.py decode -N 16

Denoising filter = no_filter
main Namespace(debug=False, subparser_name='decode', video_input='C:/tmp\\encoded', video_output='C:/tmp\\decoded', transform='2D-DCT', number_of_frames=16, gop_size=16, temporal_levels=4, block_size=16, search_range=16, wavelet_type='5/3', block_size_DCT=8, color_transform='YCoCg', perceptual_quantization=False, disable_subbands=False, quantizer='deadzone', QSS=32, filter='no_filter', entropy_image_codec='TIFF', encoded='C:/tmp\\encoded', decoded='C:/tmp\\decoded.png', func=<function decode at 0x000001E6D51409A0>)


(INFO) MCTF: MCTF Codec initialized:
(INFO) MCTF:   GOP size: 16
(INFO) MCTF:   Temporal levels: 4
(INFO) MCTF:   Block size: 16
(INFO) MCTF:   Search range: 16
(INFO) MCTF:   Wavelet type: 5/3
(INFO) MCTF:   Spatial transform: 2D-DCT
(INFO) MCTF: MCTF Decoding C:/tmp\encoded
(INFO) MCTF: Decoding 8 frames of size 352x288
(INFO) MCTF:   1 GOPs
(INFO) MCTF: Decoding GOP 0
(INFO) MCTF:   Decoding L and H frames
(INFO) MCTF:   Inverse temporal filtering with 9/7 wavelet
(INFO) temporal_filtering: Inverse temporal filtering: 4 L frames, 4 H frames
(INFO) temporal_filtering: Inverse temporal filtering complete: 8 frames
(INFO) MCTF: Decoded frame 0 to C:/tmp\decoded_0000.png
(INFO) MCTF: Decoded frame 1 to C:/tmp\decoded_0001.png
(INFO) MCTF: Decoded frame 2 to C:/tmp\decoded_0002.png
(INFO) MCTF: Decoded frame 3 to C:/tmp\decoded_0003.png
(INFO) MCTF: Decoded frame 4 to C:/tmp\decoded_0004.png
(INFO) MCTF: Decoded frame 5 to C:/tmp\decoded_0005.png
(INFO) MCTF: Decoded frame 6 to C:/tmp\de

### 3.5 Summary of Parameters

| Parameter | Description | Default | Recommended Values |
|-----------|-------------|---------|--------------------|
| `-N` | Number of frames | 30 | Depends on video length |
| `--gop_size` | Frames per GOP | 16 | 8, 16, 32 (power of 2) |
| `--block_size` | Motion estimation block size | 16 | 8, 16, 32 |
| `--search_range` | Motion search range in pixels | 16 | 8, 16, 32, 64 |
| `--wavelet_type` | Temporal wavelet | 5/3 | haar, 5/3, 9/7 |
| `-T` | Spatial transform | MPNG | MPNG, 2D-DCT, 2D-DWT |